# Importing libraries

In [2]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from sklearn import preprocessing
from itertools import islice
import seaborn as sns
from sklearn.cluster import KMeans

### Ignoring warnings

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
dataFrame = pd.read_csv('../data/week1_challenge.csv', na_values=['?', None])